In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import pickle


In [2]:
call = pd.read_csv("../data/Train_call.txt", sep='\t')

call

,Chromosome,Start,End,Nclone,Array.129,Array.34,Array.67,Array.24,Array.22,Array.36,...,Array.64,Array.89,Array.30,Array.35,Array.93,Array.10,Array.123,Array.100,Array.134,Array.130
0,1,2927,43870,3,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,-1,0
1,1,85022,216735,4,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,-1,0
2,1,370546,372295,4,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,-1,0
3,1,471671,786483,5,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,-1,0
4,1,792533,907406,13,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,23,153062077,153452633,57,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2830,23,153466463,153491568,4,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1
2831,23,153504394,153933426,55,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1
2832,23,153938998,153989329,5,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1


In [3]:
clin = pd.read_csv("../data/Train_clinical.txt", sep='\t')

clin

,Sample,Subgroup
0,Array.129,HER2+
1,Array.34,HR+
2,Array.67,HR+
3,Array.24,Triple Neg
4,Array.22,Triple Neg
...,...,...
95,Array.10,HER2+
96,Array.123,HR+
97,Array.100,HR+
98,Array.134,HR+


In [4]:
X=call[clin[clin['Subgroup']!='HER2+']['Sample']].T

X

,0,1,2,3,4,5,6,7,8,9,...,2824,2825,2826,2827,2828,2829,2830,2831,2832,2833
Array.34,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.67,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.24,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
Array.22,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.36,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.35,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.93,0,0,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.123,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.100,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [5]:
y = clin[clin['Subgroup']!='HER2+'].set_index('Sample')['Subgroup']

y

Sample
Array.34            HR+
Array.67            HR+
Array.24     Triple Neg
Array.22     Triple Neg
Array.36            HR+
                ...    
Array.35            HR+
Array.93     Triple Neg
Array.123           HR+
Array.100           HR+
Array.134           HR+
Name: Subgroup, Length: 68, dtype: object

In [6]:
folds_file = '../data/folds.pickle'
with open(folds_file, 'rb') as fid:
    outer_cross_val = pickle.load(fid)


# Baseline
Running xgboost on all regions. We reduce depth of trees in xgboost to make sure we get the simplest model since we have almost no data.

In [7]:
X_train_list, y_train_list = [], []
X_val_list, y_val_list = [], []

for split in outer_cross_val:
    inner_cross_val, test_index = split
    for train_index, val_index in inner_cross_val:
        X_train, y_train = X.iloc[train_index,:], y[train_index]
        X_val, y_val = X.iloc[val_index,:], y[val_index]

        X_train_list.append(X_train)
        y_train_list.append(y_train)

        X_val_list.append(X_val)
        y_val_list.append(y_val)

In [8]:
accuracy_list = []
av_logloss_list = []
for (X_train, y_train, X_val, y_val) in zip(X_train_list, y_train_list, X_val_list, y_val_list):
    model = XGBClassifier(max_depth=1, objective='binary:logistic', random_state=12345)
    model.fit(X_train, y_train=='HR+')

    y_pred = model.predict_proba(X_val)[:,1]
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_val=='HR+', predictions)
    av_logloss = log_loss(y_true=y_val=='HR+',labels=[True,False], y_pred=y_pred)

    accuracy_list.append(accuracy)
    av_logloss_list.append(av_logloss)

print(np.mean(accuracy_list),np.std(accuracy_list))
print(np.mean(av_logloss_list),np.std(av_logloss_list))


0.7738095238095238 0.16566022654094245
0.5308816787381823 0.2994168117777798


In [9]:
X_chr=call[clin[clin['Subgroup']!='HER2+']['Sample']].copy()
X_chr['Chromosome']=call['Chromosome']
X_chr = X_chr.groupby('Chromosome').sum()
X_chr=X_chr.T

X_chr

Chromosome,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
Array.34,4,2,-38,-1,-38,-119,3,58,-41,14,...,-23,5,-1,-8,-29,-48,64,0,-4,57
Array.67,-53,0,-13,-38,-31,-90,0,78,2,-28,...,-28,-71,-52,-301,-3,-22,2,0,3,79
Array.24,-130,-35,-42,-70,-34,-120,-43,81,-43,-60,...,-74,-50,-4,85,35,-2,0,-1,21,33
Array.22,73,0,60,-39,-18,-26,2,60,0,3,...,0,-4,-55,-3,0,-1,0,0,-7,66
Array.36,-9,-34,-3,-4,0,-7,-1,-91,-4,-5,...,-31,-4,-5,-102,0,-14,-9,36,-57,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.35,-1,0,26,-4,0,-55,40,0,2,0,...,-1,68,1,-4,0,-2,0,0,-56,69
Array.93,173,83,18,-6,-3,178,79,-114,106,112,...,3,6,2,33,58,70,2,39,7,83
Array.123,22,-25,-21,-3,-2,-58,8,105,-34,0,...,-39,-16,-27,100,-1,-22,55,-38,-19,41
Array.100,1,0,-3,-1,-5,0,-1,2,1,0,...,-5,0,-1,2,0,0,0,0,-1,73


# Baseline feature engineering
Get a sum of features across chromosomes and run xgboost. Slightly better accuracy and lower performance variance on validation set.

In [10]:
# X_train_list, y_train_list = [], []
# X_val_list, y_val_list = [], []
#
# for split in outer_cross_val:
#     inner_cross_val, test_index = split
#     for train_index, val_index in inner_cross_val:
#         X_train, y_train = X_chr.iloc[train_index,:], y[train_index]
#         X_val, y_val = X_chr.iloc[val_index,:], y[val_index]
#
#         X_train_list.append(X_train)
#         y_train_list.append(y_train)
#
#         X_val_list.append(X_val)
#         y_val_list.append(y_val)

In [11]:
# accuracy_list = []
# av_logloss_list = []
#
# for (X_train, y_train, X_val, y_val) in zip(X_train_list, y_train_list, X_val_list, y_val_list):
#     model = XGBClassifier(max_depth=1, objective='binary:logistic', random_state=12345)
#     model.fit(X_train, y_train=='HR+')
#
#     y_pred = model.predict_proba(X_val)[:,1]
#     predictions = [round(value) for value in y_pred]
#     accuracy = accuracy_score(y_val=='HR+', predictions)
#     av_logloss = log_loss(y_true=y_val=='HR+',labels=[True,False], y_pred=y_pred)
#
#     accuracy_list.append(accuracy)
#     av_logloss_list.append(av_logloss)
#
# print(np.mean(accuracy_list),np.std(accuracy_list))
# print(np.mean(av_logloss_list),np.std(av_logloss_list))

Just run everything lol

In [23]:
import joblib
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bayes",
    "Logistic Regression",
    "Ridge Regression",
    "Lasso Regression",
    "ElasticNet Regression",
    "XGBoost",
    "Light Gradient Boosting Machine",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, probability=True,random_state=12345),
    SVC(gamma=2, C=1, probability=True,random_state=12345),
    GaussianProcessClassifier(1.0 * RBF(1.0),random_state=12345),
    DecisionTreeClassifier(max_depth=5,random_state=12345),
    RandomForestClassifier(random_state=12345),
    AdaBoostClassifier(random_state=12345),
    GaussianNB(),
    LogisticRegression(penalty='none',solver='saga', max_iter=10000,random_state=12345),
    LogisticRegression(penalty='l1',solver='saga', max_iter=10000,random_state=12345),
    LogisticRegression(penalty='l2',solver='saga', max_iter=10000,random_state=12345),
    LogisticRegression(penalty = 'elasticnet',solver='saga',
                       l1_ratio=0.5, max_iter=10000,random_state=12345),
    XGBClassifier(objective='binary:logistic', random_state=12345),
    LGBMClassifier(random_state=12345)
]

def model_results(name,model):

    accuracy_list = []
    av_logloss_list = []

    for (X_train, y_train, X_val, y_val) in zip(X_train_list, y_train_list, X_val_list, y_val_list):
        model.fit(X_train, np.float32(y_train=='HR+'))

        y_pred = model.predict_proba(X_val)[:,1]
        predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(y_val=='HR+', predictions)
        av_logloss = log_loss(y_true=y_val=='HR+',labels=[True,False], y_pred=y_pred)

        accuracy_list.append(accuracy)
        av_logloss_list.append(av_logloss)
    print('{} & {:.3} ({:.3}) & {:.3} ({:.3}) \\\\'.format(
        name,
        np.mean(accuracy_list),np.std(accuracy_list),
        np.mean(av_logloss_list),np.std(av_logloss_list))
    )
    print(r'\cline{1-3}')
    joblib.dump(model, f'{name}.pkl')
    # print('Results for {} model'.format(name))
    # print('Accuracy mean: {}, std: {}'.format(np.mean(accuracy_list),np.std(accuracy_list)))
    # print('Logloss mean: {}, std: {}'.format(np.mean(av_logloss_list),np.std(av_logloss_list)))

for name, model in zip(names,classifiers):
    model_results(name,model)

Nearest Neighbors & 0.541 (0.17) & 2.26 (2.76) \\
\cline{1-3}
Linear SVM & 0.765 (0.158) & 0.516 (0.201) \\
\cline{1-3}
RBF SVM & 0.359 (0.128) & 0.706 (0.0209) \\
\cline{1-3}


KeyboardInterrupt: 

## Generate results for validation_call

In [60]:
Validation_call = pd.read_csv("../data/Validation_call.txt", sep='\t')
Validation_call

,Chromosome,Start,End,Nclone,Array.1,Array.61,Array.70,Array.14,Array.91,Array.58,...,Array.120,Array.108,Array.81,Array.126,Array.92,Array.160,Array.66,Array.83,Array.128,Array.63
0,1,2927,43870,3,0,0,1,-1,0,1,...,0,0,0,0,0,0,0,0,-1,0
1,1,85022,216735,4,0,0,1,-1,0,1,...,0,0,0,0,0,0,-1,0,-1,0
2,1,370546,372295,4,0,0,1,-1,0,-1,...,0,0,0,0,0,0,-1,0,-1,0
3,1,471671,786483,5,0,0,1,-1,0,1,...,0,0,0,0,0,0,-1,0,-1,0
4,1,792533,907406,13,0,0,1,-1,0,1,...,0,0,0,0,0,0,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,23,153062077,153452633,57,1,1,2,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2830,23,153466463,153491568,4,1,0,2,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2831,23,153504394,153933426,55,1,2,2,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2832,23,153938998,153989329,5,1,2,2,1,1,1,...,1,2,1,1,1,1,1,1,1,1


In [65]:
HER2_list = []
HER2_row = Validation_call.loc[2184]

for i,v in HER2_row.iteritems():
    if v == 2:
        HER2_list.append(i)
        Validation_call.drop([i], axis=1, inplace=True)
print(HER2_list)
Validation_call

[]


,Chromosome,Start,End,Nclone,Array.1,Array.91,Array.58,Array.140,Array.20,Array.133,...,Array.45,Array.155,Array.26,Array.108,Array.126,Array.92,Array.160,Array.66,Array.83,Array.63
0,1,2927,43870,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,85022,216735,4,0,0,1,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
2,1,370546,372295,4,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
3,1,471671,786483,5,0,0,1,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
4,1,792533,907406,13,0,0,1,0,0,0,...,0,0,0,0,0,0,0,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,23,153062077,153452633,57,1,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
2830,23,153466463,153491568,4,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2831,23,153504394,153933426,55,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2832,23,153938998,153989329,5,1,1,1,1,2,0,...,1,1,1,2,1,1,1,1,1,1


In [53]:
Validation_call.drop(['Chromosome','Start','End','Nclone'], axis=1, inplace=True)

X_test = Validation_call.T
X_test

,0,1,2,3,4,5,6,7,8,9,...,2824,2825,2826,2827,2828,2829,2830,2831,2832,2833
Array.1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,2,1,1,1,1,1
Array.91,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.58,1,1,-1,1,1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
Array.140,0,0,0,0,0,0,0,-1,-1,-1,...,1,1,1,1,0,1,1,1,1,1
Array.20,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,1
Array.133,0,0,0,0,0,1,1,1,0,0,...,2,1,1,2,2,2,0,0,0,0
Array.84,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
Array.44,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.41,0,0,0,0,0,0,0,0,-1,-1,...,1,1,1,1,1,1,1,1,1,1
Array.29,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,1


In [54]:
def read_models(model_name):
    rd = joblib.load(f'{model_name}.pkl')
    return rd

In [55]:
results_dict = {"Sample":[],"Subgroup":[]}
array_names = list(X_test.index)
y_text_pred = read_models('Ridge Regression').predict_proba(X_test)[:,1]
test_predictions = [round(value) for value in y_text_pred]

for array, result in zip(array_names,test_predictions):
    results_dict["Sample"].append(array)
    if result:
        results_dict["Subgroup"].append("HR+")
    else:
        results_dict["Subgroup"].append("Triple Neg")

for array in HER2_list:
    results_dict["Sample"].append(array)
    results_dict["Subgroup"].append("HER2+")

results_dict

{'Sample': ['Array.1',
  'Array.91',
  'Array.58',
  'Array.140',
  'Array.20',
  'Array.133',
  'Array.84',
  'Array.44',
  'Array.41',
  'Array.29',
  'Array.151',
  'Array.132',
  'Array.32',
  'Array.156',
  'Array.9',
  'Array.3',
  'Array.136',
  'Array.103',
  'Array.97',
  'Array.40',
  'Array.147',
  'Array.161',
  'Array.127',
  'Array.119',
  'Array.157',
  'Array.115',
  'Array.158',
  'Array.45',
  'Array.155',
  'Array.26',
  'Array.108',
  'Array.126',
  'Array.92',
  'Array.160',
  'Array.66',
  'Array.83',
  'Array.63',
  'Array.61',
  'Array.70',
  'Array.14',
  'Array.77',
  'Array.131',
  'Array.150',
  'Array.11',
  'Array.80',
  'Array.46',
  'Array.109',
  'Array.54',
  'Array.121',
  'Array.122',
  'Array.87',
  'Array.28',
  'Array.12',
  'Array.74',
  'Array.120',
  'Array.81',
  'Array.128'],
 'Subgroup': ['HR+',
  'Triple Neg',
  'Triple Neg',
  'Triple Neg',
  'HR+',
  'HR+',
  'Triple Neg',
  'Triple Neg',
  'HR+',
  'Triple Neg',
  'HR+',
  'HR+',
  'HR+'

In [66]:

output = pd.DataFrame.from_dict(results_dict)
output.to_csv('../data/Validation_clinical.csv', index=False)